### Imports

In [1]:
import os
import flywheel
import numpy as np
import pandas as pd
import nibabel as nib
from glob import glob
from os.path import join
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Custom imports

In [3]:
### Custom helpers
import sys
sys.path.append("classes")

# from Model import Model
# from Cloud import Cloud
# from Button import Button
from Dicom import Dicom
from DicomDir import DicomDir
from helpers import *
# from Subject import Subject

### Load objects
# cloud = Cloud()
# model = Model()

### Setup

In [4]:
project = "connectome"
subject = "conn036"
time_session = "000_data_archive"
task = "conscious"
path = "/Users/pbezuhov/tmp/connectome-conn036-000_data_archive-conscious"
dicoms = DicomDir(path)

onsets = None
patient = None
is_ascending = True # dicoms.is_ascending
num_volume = 3
dicoms.cut_volumes(3);

  0%|          | 6/6930 [00:00<02:11, 52.62it/s]

loading dicoms from /Users/pbezuhov/tmp/connectome-conn036-000_data_archive-conscious...
Sorting dicoms by trigger time...


100%|██████████| 6930/6930 [00:00<00:00, 219429.63it/s]


[{'volume': 3,
  'sheets': [{'slice_loc': "-56.7191925",
    'trigger_time': "6044",
    'dicom': <Dicom.Dicom at 0x63c4a4d30>},
   {'slice_loc': "-50.74008179",
    'trigger_time': "6088",
    'dicom': <Dicom.Dicom at 0x631426320>},
   {'slice_loc': "-44.76097107",
    'trigger_time': "6133",
    'dicom': <Dicom.Dicom at 0x636c89860>},
   {'slice_loc': "-38.78186035",
    'trigger_time': "6177",
    'dicom': <Dicom.Dicom at 0x642094828>},
   {'slice_loc': "-32.80274963",
    'trigger_time': "6222",
    'dicom': <Dicom.Dicom at 0x63ccec7f0>},
   {'slice_loc': "-26.8236351",
    'trigger_time': "6266",
    'dicom': <Dicom.Dicom at 0x676370198>},
   {'slice_loc': "-20.84452057",
    'trigger_time': "6311",
    'dicom': <Dicom.Dicom at 0x642242710>},
   {'slice_loc': "-14.86540604",
    'trigger_time': "6355",
    'dicom': <Dicom.Dicom at 0x63ce28320>},
   {'slice_loc': "-8.886291504",
    'trigger_time': "6399",
    'dicom': <Dicom.Dicom at 0x6312521d0>},
   {'slice_loc': "-2.907177687",

### Generate data

In [26]:
df = _gen_data(project, task, onsets, dicoms, patient, is_ascending, num_volume)

### Features

TODO:
- Working memory
- SCL-20, other behaviorial data

Make it faster:
- buy external harddrive

In [3]:
num_features = (
    2* (len(voxel_radius(5))) + # 4 radius voxels (previous and next volume)
#     (2* ((4 * 2) + 1)) + # trigger time for 4-radius (previous and next volume)
    1 + # trigger_time for current voxel
    1 + # is_ascending
    2 + # time since last gonogo stimuli
    2 + # time since go/nogo bloack started
    6 + # time since faces stimuli
    6 + # time since faces block started
    1 + # time since keypress (any)
    2 + # gender, age
    3 + # task identifier (is_gonogo, is_(non)conscious)
    3 + # study identifier (conn, engage, rad)
    0
)
"Number of features:", num_features

('Number of features:', 1055)

### Load settings

In [5]:
tasks  = "config/tasks.json"
EPOCHS = 100

### Test run

In [8]:
project = "connectome"
subject = "conn036"
time_session = "000_data_archive"
task = "conscious"
path = "/Users/pbezuhov/tmp/connectome-conn036-000_data_archive-conscious"
dicoms = DicomDir(path)

onsets = None
patient = None
is_ascending = True # dicoms.is_ascending
num_volume = 3
# dicoms._cut_volumes(3)
_gen_data(project, task, onsets, dicoms, patient, is_ascending, num_volume)

AttributeError: 'DicomDir' object has no attribute 'shape'

### Build model

In [19]:
def build_model(num_features):
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=(num_features, 1)),
        layers.Conv1D(100, 5, activation=tf.nn.relu), ### FIXME: check the filters and kernel_size
        layers.Dense(64, activation=tf.nn.relu),
        layers.AveragePooling1D(pool_size=2),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam()

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    return model

model = build_model(num_features)

### Run

In [ ]:
### Iterate over tasks
for task in tasks:
    task_info = tasks[task]
    num_vols  = task_info["vol"]
    if "project" in task_info:
        projects = task_info["project"]
    else:
        projects = ["rad", "connectome", "engage"]
        
    ### Iterate over projects
    for project in projects:
        time_sessions = project_config["time_sessions"]
        subjects = load_project_subjects(project)
        
        ### Iterate over time_session/subject
        for time_session in time_sessions:
            for subject in subjects:
                ### Set logger info
                logger.set(project=project, subject=subject, time_session=time_session, task=task)
            
                ### Load necessary data
                patient    = Patient(project, subject, time_session)
                dicom_path = cloud.download(project, subject, time_session, task)
                button     = Button(project, subject, time_session, task)
                
                ### Check all data exists
                if not _valid_patient(patient, dicom_path, button, logger):
                    continue
                    
                ### More setup
                fmri_data = DicomDir(dicom_path)
                fmri_data.cut_volumes(fmri_data.num_volumes - num_vols)
                
                ### Modeling
                model.load(model_path)
                model.run(patient, fmri_data, button)
                model.save(model_path)